In [12]:
%load_ext autoreload
%autoreload 1
import sys
import os
from pathlib import Path
sys.path.insert(1, str(Path(os.getcwd()).resolve().parent))

import pandas as pd 
pd.set_option('display.max_columns', 150)

from project.utils.data import *
%aimport project.utils.data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
X = load_tsfresh_data()
y = load_target_data()

In [14]:
X.shape, y.shape

((1537, 354), (1537, 1))

In [8]:
### to do

# choose df columns s. t. feature selection results 

,values__count_above__t_0,"values__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.2","values__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.4","values__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.0","values__change_quantiles__f_agg_""mean""__isabs_False__qh_0.2__ql_0.0","values__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.0","values__change_quantiles__f_agg_""var""__isabs_True__qh_0.4__ql_0.0","values__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.0","values__change_quantiles__f_agg_""var""__isabs_False__qh_0.4__ql_0.0","values__change_quantiles__f_agg_""mean""__isabs_False__qh_0.4__ql_0.0",values__minimum,"values__change_quantiles__f_agg_""var""__isabs_True__qh_0.6__ql_0.0","values__change_quantiles__f_agg_""mean""__isabs_True__qh_0.2__ql_0.0",values__has_duplicate_min,values__mean_second_derivative_central,values__range_count__max_0__min_-1000000000000.0,values__range_count__max_1000000000000.0__min_0,"values__change_quantiles__f_agg_""var""__isabs_False__qh_0.2__ql_0.0","values__change_quantiles__f_agg_""var""__isabs_True__qh_0.2__ql_0.0","values__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0","values__agg_linear_trend__attr_""rvalue""__chunk_len_10__f_agg_""min""",values__last_location_of_minimum,"values__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.6","values__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""min""","values__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""min""","values__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""",values__quantile__q_0.6,values__count_below__t_0,values__number_peaks__n_1,"values__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.6","values__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""min""","values__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""min""","values__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","values__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.4","values__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2","values__fft_coefficient__attr_""angle""__coeff_24","values__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.6",values__number_crossing_m__m_-1,"values__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.6","values__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.6","values__fft_coefficient__attr_""imag""__coeff_19","values__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.6","values__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.6",values__permutation_entropy__dimension_5__tau_1,"values__fft_coefficient__attr_""real""__coeff_10","values__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.2","values__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.4","values__fft_coefficient__attr_""real""__coeff_22",values__first_location_of_minimum,"values__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.4","values__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.2","values__fft_coefficient__attr_""angle""__coeff_10","values__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.4","values__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.2","values__fft_coefficient__attr_""abs""__coeff_22",values__permutation_entropy__dimension_3__tau_1,"values__fft_coefficient__attr_""imag""__coeff_21",values__approximate_entropy__m_2__r_0.1,"values__fft_coefficient__attr_""imag""__coeff_18","values__fft_coefficient__attr_""imag""__coeff_15",values__permutation_entropy__dimension_6__tau_1,"values__fft_coefficient__attr_""angle""__coeff_16","values__fft_coefficient__attr_""abs""__coeff_25","values__fft_coefficient__attr_""real""__coeff_25",values__ar_coefficient__coeff_8__k_10,"values__fft_coefficient__attr_""imag""__coeff_22","values__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","values__fft_coefficient__attr_""abs""__coeff_19","values__fft_coefficient__attr_""angle""__coeff_6"

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error

def time_series_model_fit(target,
                          features,
                          params_grid,
                          model_class,
                          cv_window='expanding',
                          n_splits=5,
                          additional_metric=None,
                          test_size=0.2
                         ):
    # Step 1: Split time series data into train and test
    split_index = int(len(target) * (1 - test_size))
    target_train, target_test = target[:split_index], target[split_index:]
    features_train, features_test = features[:split_index], features[split_index:]

    # Step 2: Define Time Series Cross-Validation
    if cv_window == 'expanding':
        cv = TimeSeriesSplit(n_splits=n_splits if n_splits else len(target_train))
    elif cv_window == 'rolling':
        cv = TimeSeriesSplit(n_splits=n_splits)

    # Step 3: Run Grid Search CV on the train data
    grid_search = GridSearchCV(model_class, params_grid, cv=cv, scoring='neg_mean_absolute_error')
    grid_search.fit(features_train, target_train)

    best_model = grid_search.best_estimator_

    # Step 4: Evaluate the model on the test data
    predictions_test = best_model.predict(features_test)
    mae_test = mean_absolute_error(target_test, predictions_test)

    # Step 5: Fit the best model on all data
    best_model.fit(features, target)

    additional_metric_result = None
    if additional_metric:
        additional_metric_result = additional_metric(target_test, predictions_test)

    return best_model, mae_test, additional_metric_result, grid_search.best_params_



In [16]:
from catboost import  CatBoostRegressor

In [17]:
param_grid = {  
    'iterations': [100, 200, 300],  
    'learning_rate': [0.1, 1],  
    'depth': [5, 7, 8],  
    # 'l2_leaf_reg': [1, 5, 10],  
    # 'random_strength': [1, 5, 10],  
    # 'bagging_temperature': [0.5, 1, 2],  
    # 'random_seed': [0, 42, 123]  
}  
  
model = CatBoostRegressor(verbose=0) 

In [18]:
# y.to_numpy()

In [19]:
best_model, mae_test, additional_metric_result, best_params = time_series_model_fit(
    target=y,
    features=X, params_grid=param_grid, model_class=model, cv_window='rolling'
)

In [20]:
mae_test

0.04083322358146545

In [21]:
best_params

{'depth': 5, 'iterations': 200, 'learning_rate': 0.1}